# CCA: Continuity Care Assistant

**A continuity-focused "Don't Miss" alert system for primary care physicians**

Author: J.D. Laurence-Chasen

---

### The Problem

Primary care providers (PCPs) face an impossible information synthesis challenge:

- **<15 minutes** per appointment of face-to-face time with patients
- **100+ pages** of notes, labs, and imaging per patient
- **Critical patterns get missed**: slow-building disease presentations, imaging follow-ups, drug-disease interactions

> *"There is just so much data in [a patient's] chart. There's no way to review all of it before an appointment--important connections can be missed"*  
> — M.L.W, M.D., University of Washington, Interview

### Our Solution

CCA uses **MedGemma 1.5 4B** in a two-stage, patient-centered pipeline that ingests and synthesizes all of a patient's data, generating "Don't Miss" alerts that can be integrated directly into an EMR Dashboard for easy access before or during an appointment. 

**A note on novelty**: An AI-powered chart synthesis alone **is not** novel. Here, we leverage MedGemma 1.5's new functionality to perform **longitudinal image comparison** (e.g., multiple chest X-rays from different visits) to create a truly continuity-focused pipeline. When the system detects multiple images of the same modality, it automatically feeds them together into MedGemma for temporal comparison — surfacing new findings, progression, or regression that might otherwise be missed. This centers the patient, rather than any single note, test, or scan.

```
┌─────────────────────────────────────────────────────────────────┐
│              CCA: Continuity Care Assistant Pipeline            │
├─────────────────────────────────────────────────────────────────┤
│  ┌───────────────────────────────────────────────────────────┐  │
│  │                   STAGE 1: EXTRACTION                     │  │
│  │                     MedGemma 1.5 4B                       │  │
│  ├───────────────────────────────────────────────────────────|  │
│  │                                                           │  │
│  │  Clinical Notes ──→ symptoms, findings, unlisted meds     │  │
│  │  Chest X-rays   ──→ nodules, masses, abnormalities        │  │
│  │  CT/MRI Scans   ──→ 3D volume interpretation              |  |
|  |  Repeat images  ──→ longitudinal comparison               │  │
│  │  Lab Reports    ──→ structured values, trends             │  │
│  │                                                           │  │
│  └───────────────────────────────────────────────────────────┘  │
│                               ▼                                 │
│                    ┌─────────────────────┐                      │
│                    │  Aggregated Context │                      │
│                    └─────────────────────┘                      │
│                               ▼                                 │
│  ┌───────────────────────────────────────────────────────────┐  │
│  │                   STAGE 2: SYNTHESIS                      │  │
│  │                   MedGemma 1.5 4B                         │  │
│  ├───────────────────────────────────────────────────────────┤  │
│  │                                                           │  │
│  │  Cross-modal reasoning across all extracted findings      │  │
│  │  Pattern detection across longitudinal visits             │  │
│  │  Risk factor correlation and amplification                |  |
|  |  Safety rules integration (NSAID+CKD, etc.)               |  |
│  │  Alert generation with evidence chains and recommendations│  │
│  │                                                           │  │
│  └───────────────────────────────────────────────────────────┘  │
│                               ▼                                 │
│                    ┌─────────────────────┐                      │
│                    │ "Don't Miss" Alerts │                      │
│                    └─────────────────────┘                      │
└─────────────────────────────────────────────────────────────────┘
```

---
## 1. Setup

In [41]:
# Install dependencies (uncomment on first run)
#!pip install transformers torch accelerate bitsandbytes

from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
from typing import Optional
from IPython.display import HTML, display
import json
import re

# Set to True if running on GPU (Colab A100, etc.)
USE_MEDGEMMA = False

print("✓ Setup complete")

✓ Setup complete


---
## 2. Data Models & Loaders

Simple dataclasses to represent patient data and alerts and model loader

In [42]:
class AlertSeverity(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"

@dataclass
class Lab:
    name: str
    value: float
    unit: str
    date: datetime
    ref_low: Optional[float] = None
    ref_high: Optional[float] = None
    
    @property
    def is_abnormal(self) -> bool:
        if self.ref_low and self.value < self.ref_low:
            return True
        if self.ref_high and self.value > self.ref_high:
            return True
        return False

@dataclass
class Medication:
    name: str
    dose: str
    start_date: datetime
    prescriber: str

@dataclass
class Imaging:
    modality: str
    date: datetime
    indication: str
    findings: str
    impression: str
    follow_up: Optional[str] = None  # e.g., "CT chest in 3 months"
    follow_up_completed: bool = False
    image_url: Optional[str] = None  # URL to the actual image (for MedGemma inference)

@dataclass
class Note:
    date: datetime
    author: str
    note_type: str
    chief_complaint: str
    content: str

@dataclass
class Patient:
    id: str
    name: str
    age: int
    sex: str
    problems: list[str] = field(default_factory=list)
    medications: list[Medication] = field(default_factory=list)
    labs: list[Lab] = field(default_factory=list)
    imaging: list[Imaging] = field(default_factory=list)
    notes: list[Note] = field(default_factory=list)
    smoking_status: str = "never"
    family_history: list[str] = field(default_factory=list)
    
    def get_lab_trend(self, lab_name: str) -> list[Lab]:
        """Get all results for a lab test, sorted by date."""
        return sorted(
            [l for l in self.labs if l.name == lab_name],
            key=lambda x: x.date
        )
    
    def has_medication_class(self, drug_class: list[str]) -> Optional[Medication]:
        """Check if patient is on any medication in the given class."""
        for med in self.medications:
            if any(drug.lower() in med.name.lower() for drug in drug_class):
                return med
        return None

@dataclass
class Alert:
    severity: AlertSeverity
    headline: str
    summary: str
    evidence: list[str]
    recommendations: list[str]

def extract_json(text: str) -> dict:
    """Extract JSON from model output, skipping MedGemma chain-of-thought.

    MedGemma may prefix responses with <unused94>thought...reasoning...
    before the actual JSON. This uses json.raw_decode to find the last
    valid JSON object, which is the actual answer (not CoT fragments).
    """
    text = re.sub(r'<unused\d+>', '', text)
    # Prefer ```json blocks when present
    m = re.search(r'```json?\s*([\s\S]*?)\s*```', text)
    if m:
        try:
            return json.loads(m.group(1).strip())
        except json.JSONDecodeError:
            pass
    # Find last valid JSON object via raw_decode
    decoder = json.JSONDecoder()
    result = {}
    for i, ch in enumerate(text):
        if ch == '{':
            try:
                obj, _ = decoder.raw_decode(text, i)
                result = obj
            except (json.JSONDecodeError, ValueError):
                continue
    return result

# =============================================================================
# Shared MedGemma 1.5 4B loader — single instance used by all pipeline stages
# =============================================================================

def load_medgemma():
    """Load MedGemma 1.5 4B model and processor once for the entire pipeline."""
    from transformers import AutoProcessor, AutoModelForImageTextToText
    import torch
    print("Loading MedGemma 1.5 4B...")
    processor = AutoProcessor.from_pretrained("google/medgemma-1.5-4b-it")
    model = AutoModelForImageTextToText.from_pretrained(
        "google/medgemma-1.5-4b-it",
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    print(f"✓ MedGemma 1.5 4B loaded on {model.device}")
    return model, processor

MEDGEMMA_MODEL, MEDGEMMA_PROCESSOR = load_medgemma() if USE_MEDGEMMA else (None, None)

print("✓ Data models and loaders defined")

✓ Data models and loaders defined


---
## 3. Synthetic Patients

Three patients demonstrating different alert types, **sourced from real interviews with practicing primary care physicians**:

| Patient | Age | Alert Type | Clinical Scenario |
|---------|-----|------------|-------------------|
| Maria Klein | 58 | Slow-building pattern | Progressive iron deficiency anemia + weight loss + GI symptoms |
| James Morrison | 64 | Missed follow-up | Lung nodule requiring follow-up CT, never completed |
| Dorothy Williams | 72 | Drug-disease interaction | NSAID prescribed despite CKD diagnosis |

In [43]:
TODAY = datetime(2026, 2, 5)

# =============================================================================
# Chest X-ray image URLs for longitudinal comparison demo
# Public domain images from Wikimedia Commons
# =============================================================================
# Prior normal CXR (also used in MedGemma model card example)
# Attribution: Stillwaterising, CC0, via Wikimedia Commons
CXR_IMAGE_URL_PRIOR = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"

# Current CXR showing left hilar mass (from Wikipedia: Lung cancer)
# Attribution: James Heilman, MD, CC BY-SA 3.0, via Wikimedia Commons
CXR_IMAGE_URL_CURRENT = "https://upload.wikimedia.org/wikipedia/commons/a/ad/Lung_Cancer_on_Chest_X-Ray.jpg"

# =============================================================================
# PATIENT 1: Maria Klein - "The Slow-Building Pattern"
# =============================================================================
maria = Patient(
    id="MK-001",
    name="Maria Klein",
    age=58,
    sex="Female",
    problems=["Hypertension", "Hyperlipidemia", "GERD"],
    smoking_status="never",
    family_history=["Mother: breast cancer at 65", "Father: MI at 70"],
    medications=[
        Medication("Lisinopril", "10mg daily", TODAY - timedelta(days=365), "Dr. Smith"),
        Medication("Atorvastatin", "20mg daily", TODAY - timedelta(days=365), "Dr. Smith"),
        Medication("Omeprazole", "20mg daily", TODAY - timedelta(days=180), "Dr. Smith"),
    ],
    labs=[
        # Hemoglobin trend - subtle decline
        Lab("Hemoglobin", 13.2, "g/dL", TODAY - timedelta(days=240), 12.0, 16.0),
        Lab("Hemoglobin", 12.8, "g/dL", TODAY - timedelta(days=180), 12.0, 16.0),
        Lab("Hemoglobin", 12.1, "g/dL", TODAY - timedelta(days=90), 12.0, 16.0),
        Lab("Hemoglobin", 11.4, "g/dL", TODAY - timedelta(days=14), 12.0, 16.0),
        # MCV trend - becoming microcytic
        Lab("MCV", 88, "fL", TODAY - timedelta(days=240), 80, 100),
        Lab("MCV", 82, "fL", TODAY - timedelta(days=90), 80, 100),
        Lab("MCV", 76, "fL", TODAY - timedelta(days=14), 80, 100),
        # Iron studies
        Lab("Ferritin", 18, "ng/mL", TODAY - timedelta(days=14), 20, 200),
        Lab("Iron", 35, "mcg/dL", TODAY - timedelta(days=14), 60, 170),
        # Stool guaiac
        Lab("Stool Guaiac", 1, "pos", TODAY - timedelta(days=7), None, None),  # 1 = positive
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=240),
            author="Dr. Smith",
            note_type="Annual Physical",
            chief_complaint="Annual exam",
            content="""58yo F presents for annual physical. Reports feeling 'a bit more tired than usual' 
over past few weeks, attributes to stress at work. Denies chest pain, SOB, palpitations. 
Weight 145 lbs (stable). BP 128/82. Labs ordered including CBC, CMP, lipids. 
Due for colonoscopy - discussed, patient wants to wait until after busy season at work."""
        ),
        Note(
            date=TODAY - timedelta(days=120),
            author="Dr. Smith",
            note_type="Office Visit",
            chief_complaint="Fatigue, unintentional weight loss",
            content="""Patient returns reporting persistent fatigue despite improved sleep. 
Notes clothes fitting looser - reports 8 lb weight loss over 4 months without trying.
Appetite slightly decreased. Occasional bloating after meals. Denies N/V, melena, hematochezia.
Weight today 137 lbs (down from 145). Discussed colonoscopy again - scheduled for next month."""
        ),
        Note(
            date=TODAY - timedelta(days=14),
            author="Dr. Smith",
            note_type="Office Visit",
            chief_complaint="Constipation, bloating",
            content="""Reports worsening constipation over past 3 weeks, alternating with loose stools.
Persistent bloating and early satiety. Continued weight loss - now 132 lbs.
Labs show Hgb 11.4 (was 12.8 six months ago), MCV 76, Ferritin 18.
Stool guaiac positive. Iron deficiency anemia with GI symptoms - colonoscopy was cancelled 
by patient due to scheduling conflict. Need to reschedule urgently."""
        ),
    ],
)

# =============================================================================
# PATIENT 2: James Morrison - "The Missed Follow-up"
# =============================================================================
james = Patient(
    id="JM-002",
    name="James Morrison",
    age=64,
    sex="Male",
    problems=["Hypertension", "Type 2 Diabetes", "Former smoker"],
    smoking_status="former (20 pack-years, quit 5 years ago)",
    family_history=["Father: lung cancer, died at 62", "Mother: COPD"],
    medications=[
        Medication("Metformin", "1000mg BID", TODAY - timedelta(days=730), "Dr. Johnson"),
        Medication("Amlodipine", "5mg daily", TODAY - timedelta(days=365), "Dr. Johnson"),
    ],
    labs=[
        Lab("Calcium", 10.2, "mg/dL", TODAY - timedelta(days=180), 8.5, 10.5),
        Lab("Calcium", 10.8, "mg/dL", TODAY - timedelta(days=14), 8.5, 10.5),
        Lab("PTH", 18, "pg/mL", TODAY - timedelta(days=14), 15, 65),  # Low-normal (suppressed)
        Lab("HbA1c", 7.2, "%", TODAY - timedelta(days=90), None, 7.0),
    ],
    imaging=[
        # Prior CXR - normal baseline (from annual physical ~15 months ago)
        Imaging(
            modality="Chest X-ray",
            date=TODAY - timedelta(days=460),
            indication="Routine screening, annual physical",
            findings="Lungs clear bilaterally. No masses, nodules, or infiltrates. Heart size normal. No pleural effusion.",
            impression="Normal chest radiograph. No acute cardiopulmonary abnormality.",
            image_url=CXR_IMAGE_URL_PRIOR,
        ),
        # Current CXR - left hilar mass found on pre-op imaging
        Imaging(
            modality="Chest X-ray",
            date=TODAY - timedelta(days=130),
            indication="Pre-operative clearance for knee arthroscopy",
            findings="Large left hilar mass. No prior imaging available for comparison. Lungs otherwise clear. No pleural effusion.",
            impression="Left hilar mass, suspicious for malignancy. Given smoking history, recommend CT chest with contrast for further evaluation. Pulmonology referral advised.",
            follow_up="CT chest with contrast",
            follow_up_completed=False,
            image_url=CXR_IMAGE_URL_CURRENT,
        ),
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=130),
            author="Dr. Johnson",
            note_type="Pre-op Visit",
            chief_complaint="Pre-op clearance",
            content="""64yo M with HTN, T2DM, former smoker here for pre-op clearance for knee arthroscopy.
CXR shows left hilar mass - new finding, no prior imaging available for comparison.
Given smoking history (20 pack-years) and family history of lung cancer, this is highly
concerning. CT chest with contrast ordered urgently. Pulmonology referral placed.
Discussed with patient - he understands need for urgent follow-up.
Cleared for surgery from cardiac standpoint."""
        ),
        Note(
            date=TODAY - timedelta(days=45),
            author="Dr. Patel",
            note_type="Urgent Care",
            chief_complaint="Upper respiratory infection",
            content="""Patient presents with 5 days of cough, congestion, low-grade fever.
No SOB, chest pain. Lungs with scattered rhonchi, no wheezes. Likely viral URI.
Supportive care recommended. Return if worsening."""
        ),
        Note(
            date=TODAY - timedelta(days=14),
            author="Dr. Johnson",
            note_type="Office Visit",
            chief_complaint="Routine diabetes follow-up",
            content="""Here for DM follow-up. A1c 7.2% - slightly above goal. Increase metformin?
Incidentally noted calcium slightly elevated at 10.8. PTH 18 (low-normal).
Will monitor calcium. Continue current regimen for now."""
        ),
    ],
)

# =============================================================================
# PATIENT 3: Dorothy Williams - "The Drug-Disease Interaction"
# =============================================================================
dorothy = Patient(
    id="DW-003",
    name="Dorothy Williams",
    age=72,
    sex="Female",
    problems=["CKD Stage 3a", "Hypertension", "Osteoarthritis", "Type 2 Diabetes"],
    smoking_status="never",
    family_history=["Brother: ESRD on dialysis"],
    medications=[
        Medication("Losartan", "50mg daily", TODAY - timedelta(days=730), "Dr. Chen"),
        Medication("Metformin", "500mg BID", TODAY - timedelta(days=730), "Dr. Chen"),
        Medication("Acetaminophen", "650mg TID PRN", TODAY - timedelta(days=365), "Dr. Chen"),
        Medication("Meloxicam", "15mg daily", TODAY - timedelta(days=18), "Dr. Roberts (Ortho)"),  # NEW!
    ],
    labs=[
        # Creatinine trend - stable until recently
        Lab("Creatinine", 1.3, "mg/dL", TODAY - timedelta(days=365), 0.6, 1.2),
        Lab("Creatinine", 1.3, "mg/dL", TODAY - timedelta(days=180), 0.6, 1.2),
        Lab("Creatinine", 1.4, "mg/dL", TODAY - timedelta(days=90), 0.6, 1.2),
        Lab("Creatinine", 1.8, "mg/dL", TODAY - timedelta(days=7), 0.6, 1.2),  # Acute rise!
        # eGFR trend
        Lab("eGFR", 52, "mL/min", TODAY - timedelta(days=365), 60, None),
        Lab("eGFR", 51, "mL/min", TODAY - timedelta(days=180), 60, None),
        Lab("eGFR", 48, "mL/min", TODAY - timedelta(days=90), 60, None),
        Lab("eGFR", 35, "mL/min", TODAY - timedelta(days=7), 60, None),  # Significant drop!
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=365),
            author="Dr. Chen",
            note_type="Annual Visit",
            chief_complaint="Annual exam",
            content="""72yo F with CKD 3a, HTN, T2DM, OA for annual exam. CKD stable - eGFR 52.
IMPORTANT: Avoid NSAIDs due to CKD - discussed with patient. Continue acetaminophen PRN for OA pain.
BP well controlled on losartan. A1c 6.8 - at goal."""
        ),
        Note(
            date=TODAY - timedelta(days=21),
            author="Dr. Roberts",
            note_type="Orthopedics Consult",
            chief_complaint="Knee pain",
            content="""72yo F referred for right knee pain. X-ray shows moderate OA with joint space narrowing.
Conservative management recommended. Starting meloxicam 15mg daily for inflammation.
Physical therapy referral placed. Follow-up in 6 weeks."""
        ),
        Note(
            date=TODAY - timedelta(days=7),
            author="Dr. Chen",
            note_type="Lab Review",
            chief_complaint="Lab follow-up",
            content="""Routine CKD monitoring labs. Creatinine 1.8 (was 1.4 three months ago) - 
concerning rise. eGFR dropped to 35 from 48. Patient reports knee feeling better.
Need to review medication list for nephrotoxins."""
        ),
    ],
)

PATIENTS = {"maria": maria, "james": james, "dorothy": dorothy}
print(f"✓ Loaded {len(PATIENTS)} synthetic patients")

✓ Loaded 3 synthetic patients


### 3b. Expected Outputs (Ground Truth)

These are the clinician-verified expected outputs for each patient. They serve as ground truth for evaluating MedGemma's performance and as fallback outputs when running without a GPU.

In [44]:
# =============================================================================
# GROUND TRUTH: Clinician-verified expected outputs for each patient
# Used for (a) mock mode fallback and (b) evaluation of MedGemma outputs
# =============================================================================

# ── Note Extractions ────────────────────────────────────────────────
# Key: (patient_id, note_index) → ExtractedNote fields

EXPECTED_NOTE_EXTRACTIONS = {
    # --- Maria Klein ---
    ("MK-001", 0): {  # Annual Physical
        "symptoms": [
            {"symptom": "fatigue", "duration": "weeks", "severity": "moderate", "trend": "persistent"},
        ],
        "findings": [],
        "follow_ups": [
            {"item": "colonoscopy", "timeframe": "scheduled", "completed": False},
        ],
        "red_flags": [],
    },
    ("MK-001", 1): {  # Office Visit - weight loss
        "symptoms": [
            {"symptom": "fatigue", "duration": "weeks", "severity": "moderate", "trend": "persistent"},
            {"symptom": "unintentional weight loss", "duration": "months", "severity": "concerning", "trend": "progressive"},
            {"symptom": "bloating", "duration": "weeks", "severity": "moderate", "trend": "worsening"},
        ],
        "findings": [
            {"finding": "anemia", "value": "Hgb declining", "interpretation": "concerning"},
        ],
        "follow_ups": [
            {"item": "colonoscopy", "timeframe": "scheduled", "completed": False},
        ],
        "red_flags": [
            "Constitutional symptoms with anemia - malignancy workup needed",
        ],
    },
    ("MK-001", 2): {  # Office Visit - constipation/bloating
        "symptoms": [
            {"symptom": "constipation", "duration": "weeks", "severity": "moderate", "trend": "worsening"},
            {"symptom": "bloating", "duration": "weeks", "severity": "moderate", "trend": "worsening"},
            {"symptom": "unintentional weight loss", "duration": "months", "severity": "concerning", "trend": "progressive"},
        ],
        "findings": [
            {"finding": "anemia", "value": "Hgb declining", "interpretation": "concerning"},
            {"finding": "stool guaiac", "value": "positive", "interpretation": "GI blood loss"},
        ],
        "follow_ups": [
            {"item": "colonoscopy", "timeframe": "scheduled", "completed": False},
        ],
        "red_flags": [
            "Constitutional symptoms with anemia - malignancy workup needed",
        ],
    },
    # --- James Morrison ---
    ("JM-002", 0): {  # Pre-op Visit
        "symptoms": [],
        "findings": [
            {"finding": "left hilar mass", "value": "suspicious for malignancy", "interpretation": "urgent workup needed"},
        ],
        "follow_ups": [
            {"item": "CT chest follow-up", "timeframe": "urgent", "completed": False},
        ],
        "red_flags": [
            "Lung mass in high-risk patient — urgent evaluation needed",
        ],
    },
    ("JM-002", 1): {  # Urgent Care
        "symptoms": [
            {"symptom": "cough", "duration": "days", "severity": "mild", "trend": "stable"},
        ],
        "findings": [],
        "follow_ups": [
            {"item": "imaging follow-up recommended", "timeframe": "per note", "completed": False},
        ],
        "red_flags": [],
    },
    ("JM-002", 2): {  # Diabetes follow-up
        "symptoms": [],
        "findings": [],
        "follow_ups": [],
        "red_flags": [],
    },
    # --- Dorothy Williams ---
    ("DW-003", 0): {  # Annual Visit
        "symptoms": [],
        "findings": [],
        "follow_ups": [],
        "red_flags": [],
    },
    ("DW-003", 1): {  # Ortho Consult
        "symptoms": [
            {"symptom": "joint pain", "duration": "chronic", "severity": "moderate", "trend": "stable"},
        ],
        "findings": [],
        "follow_ups": [
            {"item": "imaging follow-up recommended", "timeframe": "per note", "completed": False},
        ],
        "red_flags": [],
    },
    ("DW-003", 2): {  # Lab Review
        "symptoms": [],
        "findings": [
            {"finding": "acute kidney injury", "value": "Cr rising", "interpretation": "concerning"},
        ],
        "follow_ups": [],
        "red_flags": [
            "NSAID prescribed to patient with renal disease",
        ],
    },
}

# ── Image Analyses ──────────────────────────────────────────────────
# Key: (patient_id, image_index) → ImageAnalysis fields

EXPECTED_IMAGE_ANALYSES = {
    ("JM-002", 0): {  # Prior normal CXR
        "findings": [
            {"finding": "no acute abnormality", "location": "chest", "confidence": 0.92},
        ],
        "follow_up_needed": False,
        "urgency": "routine",
    },
    ("JM-002", 1): {  # Current CXR with hilar mass
        "findings": [
            {"finding": "pulmonary mass", "location": "lung", "confidence": 0.91},
        ],
        "follow_up_needed": True,
        "urgency": "urgent",
    },
}

# ── Longitudinal Comparisons ───────────────────────────────────────
# Key: (patient_id, modality)

EXPECTED_LONGITUDINAL_COMPARISONS = {
    ("JM-002", "Chest X-ray"): {
        "comparison_findings": [
            {
                "finding": "New pulmonary mass, not present on prior imaging",
                "change": "new",
                "clinical_significance": "high",
            },
        ],
        "overall_assessment": (
            "Comparison of Chest X-ray images 330 days apart reveals "
            "1 new finding(s) not present on prior study. "
            "Clinical correlation and further workup recommended."
        ),
        "urgency": "urgent",
    },
}

# ── Synthesized Alerts ──────────────────────────────────────────────
# Key: patient_id → list[Alert fields]

EXPECTED_ALERTS = {
    "MK-001": [{
        "severity": "critical",
        "headline": "Iron Deficiency Anemia with Alarm Features — Rule Out GI Malignancy",
        "summary": (
            "Progressive microcytic anemia over 8 months with unintentional weight loss (13 lbs), "
            "new GI symptoms, and positive stool guaiac. "
            "In a 58-year-old, this constellation strongly suggests occult GI blood loss."
        ),
        "evidence": [
            "Note extraction: 'fatigue' (3 visits), 'weight loss' (2 visits), 'bloating' (recent)",
            "Lab trends: Hemoglobin 13.2→11.4 g/dL (decreasing), MCV 88→76 fL (microcytic)",
            "Lab flags: Iron deficiency anemia pattern detected",
            "Follow-up tracking: Colonoscopy deferred multiple times",
            "Stool guaiac: POSITIVE",
        ],
        "recommendations": [
            "Schedule colonoscopy URGENTLY — do not allow further deferral",
            "GI referral for expedited workup",
            "Consider CT abdomen/pelvis if colonoscopy delayed",
            "Check CEA level",
        ],
    }],
    "JM-002": [{
        "severity": "critical",
        "headline": "Left Hilar Mass — Confirmed NEW on Longitudinal Comparison, CT Overdue",
        "summary": (
            "Longitudinal comparison of two chest X-rays confirms the left hilar "
            "mass is NEW — it was not present on the prior CXR ~11 months earlier. "
            "Follow-up CT with contrast was never completed (40+ days overdue). Combined with new "
            "hypercalcemia with suppressed PTH, this demands urgent evaluation for malignancy."
        ),
        "evidence": [
            "Longitudinal comparison: Left hilar mass is NEW — not present on prior CXR",
            "Image analysis: Left hilar mass, suspicious for malignancy (91% confidence)",
            "Follow-up tracking: 'CT chest with contrast' — NOT COMPLETED (40+ days overdue)",
            "Risk factors: 20 pack-year smoking, father died of lung cancer at 62",
            "Lab trends: Calcium 10.2→10.8 mg/dL (increasing)",
            "Lab flags: Hypercalcemia with suppressed PTH — evaluate for malignancy",
        ],
        "recommendations": [
            "Order CT chest with contrast URGENTLY",
            "Contact patient directly — do not rely on mailed reminder",
            "Pulmonology referral given risk factors",
            "Complete hypercalcemia workup: Vitamin D, PTHrP, SPEP",
        ],
    }],
    "DW-003": [{
        "severity": "high",
        "headline": "Accelerated CKD Progression — Evaluate for Nephrotoxic Cause",
        "summary": (
            "eGFR dropped from 48 to 35 mL/min over 3 months, "
            "significantly faster than prior stable trajectory. Temporal correlation with "
            "recent orthopedics visit suggests medication-induced injury."
        ),
        "evidence": [
            "Lab trends: eGFR 52→51→48→35 mL/min (accelerated decline)",
            "Lab flags: Acute kidney injury (>20% Cr rise)",
            "Note extraction: Recent orthopedics consult with new prescriptions",
            "Temporal correlation: Decline began after orthopedics visit",
        ],
        "recommendations": [
            "Review all medications for nephrotoxins",
            "Hold any identified nephrotoxins immediately",
            "Recheck creatinine in 1 week",
            "Nephrology referral if no reversible cause identified",
        ],
    }],
}

print("✓ Ground truth definitions loaded")

✓ Ground truth definitions loaded


---
## 4. Display Helpers

In [45]:
def show_patient(patient: Patient):
    """Display patient summary card."""
    problems = ", ".join(patient.problems[:4])
    meds = ", ".join(m.name.split()[0] for m in patient.medications[:4])
    html = f"""
    <div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); 
                border-radius: 10px; padding: 20px; color: white; margin: 15px 0; font-family: system-ui;">
        <div style="display: flex; justify-content: space-between; align-items: start;">
            <div>
                <h2 style="margin: 0; font-size: 24px;">{patient.name}</h2>
                <p style="margin: 5px 0; opacity: 0.8;">{patient.age}yo {patient.sex} · MRN: {patient.id}</p>
            </div>
            <div style="text-align: right; font-size: 13px; opacity: 0.7;">
                <div>🚬 {patient.smoking_status}</div>
            </div>
        </div>
        <hr style="border: none; border-top: 1px solid rgba(255,255,255,0.2); margin: 15px 0;">
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 14px;">
            <div><strong>Problems:</strong> {problems}</div>
            <div><strong>Medications:</strong> {meds}</div>
        </div>
    </div>
    """
    display(HTML(html))

def show_alert(alert: Alert):
    """Display an alert card."""
    colors = {
        AlertSeverity.CRITICAL: ("#dc3545", "🔴"),
        AlertSeverity.HIGH: ("#fd7e14", "🟠"),
        AlertSeverity.MEDIUM: ("#ffc107", "🟡"),
    }
    color, icon = colors.get(alert.severity, ("#6c757d", "⚪"))
    
    evidence_html = "\n".join(f"<li>{e}</li>" for e in alert.evidence)
    recs_html = "\n".join(f"<li><strong>{r}</strong></li>" for r in alert.recommendations)
    
    html = f"""
    <div style="border-left: 4px solid {color}; background: #fff; border-radius: 8px; 
                padding: 16px; margin: 15px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.1); font-family: system-ui; max-width: 500px;">
        <div style="display: flex; align-items: center; gap: 10px; margin-bottom: 10px;">
            <span style="font-size: 20px;">{icon}</span>
            <span style="background: {color}; color: white; padding: 3px 10px; border-radius: 4px; 
                        font-size: 12px; font-weight: 600; text-transform: uppercase;">Don't Miss: {alert.severity.value}</span>
        </div>
        <h3 style="margin: 0 0 12px 0; color: #1a1a1a;">{alert.headline}</h3>
        
        <details style="margin-bottom: 12px;">
            <summary style="cursor: pointer; font-weight: 600; color: #333;">📋 Evidence ({len(alert.evidence)} items)</summary>
            <ul style="margin-top: 8px; color: #666; line-height: 1.6;">{evidence_html}</ul>
        </details>
        
        <details style="margin-bottom: 0;">
            <summary style="cursor: pointer; font-weight: 600; color: #0066cc;">⚡ Recommended Actions ({len(alert.recommendations)})</summary>
            <div style="background: #f0f7ff; padding: 12px; border-radius: 6px; margin-top: 8px;">
                <ul style="margin: 0; padding-left: 20px; color: #333;">{recs_html}</ul>
            </div>
        </details>
    </div>
    """
    display(HTML(html))

print("✓ Display helpers ready")

✓ Display helpers ready


---
## 5. Stage 1: Data Extraction

MedGemma 1.5 4B handles all extraction tasks in Stage 1:

- **Clinical notes** → symptoms, findings, follow-up items
- **Medical images** → abnormalities, measurements, follow-up recommendations  
- **Lab reports** → structured values (supplemented by deterministic trend analysis)
- Note that, for the purposes of this demo, PDFs of notes/charts are not used, but could easily be added in the future.

### 5a. Clinical Note Extraction

In [46]:
# =============================================================================
# STAGE 1a: Clinical Note Extraction (MedGemma 1.5 4B)
# =============================================================================

NOTE_EXTRACTION_PROMPT = """Extract structured clinical information from this note.

NOTE:
{note_text}

Return JSON with:
{{
  "symptoms": [
    {{"symptom": "name", "duration": "timeframe", "severity": "mild/moderate/severe", "trend": "stable/worsening/improving"}}
  ],
  "clinical_findings": [
    {{"finding": "name", "value": "measurement if applicable", "interpretation": "normal/abnormal/concerning"}}
  ],
  "follow_up_items": [
    {{"item": "description", "timeframe": "when", "completed": true/false}}
  ],
  "red_flags": ["any alarming findings"]
}}

Be specific and extract actual values from the note. Respond with ONLY the JSON object, no explanation."""

@dataclass
class ExtractedNote:
    """Structured extraction from a clinical note."""
    date: datetime
    note_type: str
    symptoms: list[dict]
    findings: list[dict]
    follow_ups: list[dict]
    red_flags: list[str]

class NoteExtractor:
    """
    MedGemma 1.5 4B for clinical note extraction.
    Returns a structured summary of symptoms, clinical findings, follow-up items, and red flags.
    """

    def __init__(self, model=None, processor=None):
        self.use_medgemma = model is not None
        self.model = model
        self.processor = processor

    def extract(self, note: Note) -> ExtractedNote:
        """Extract structured information from a clinical note."""
        if self.use_medgemma:
            return self._extract_with_model(note)
        else:
            return self._mock_extract(note)

    def _extract_with_model(self, note: Note) -> ExtractedNote:
        import torch
        prompt = NOTE_EXTRACTION_PROMPT.format(note_text=note.content)

        messages = [{"role": "user", "content": [{"type": "text", "text": prompt}]}]

        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device)

        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            # Added sampling and explicit pad_token_id to silence warnings
            generation = self.model.generate(
                **inputs, 
                max_new_tokens=1500, 
                do_sample=True, 
                temperature=0.1,
                top_p=0.95,
                pad_token_id=self.processor.tokenizer.eos_token_id
            )
            generation = generation[0][input_len:]

        response = self.processor.decode(generation, skip_special_tokens=True)

        data = extract_json(response)

        # Ensure fields exist even if the model missed one
        return ExtractedNote(
            date=note.date,
            note_type=note.note_type,
            symptoms=data.get("symptoms", []),
            findings=data.get("clinical_findings", []),
            follow_ups=data.get("follow_up_items", []),
            red_flags=data.get("red_flags", [])
        )

    def set_patient(self, patient_id: str):
        """Set current patient context for ground truth lookup."""
        self._patient_id = patient_id
        self._note_index = 0

    def _mock_extract(self, note: Note) -> ExtractedNote:
        """Look up expected output from ground truth."""
        key = (getattr(self, '_patient_id', ''), getattr(self, '_note_index', 0))
        self._note_index = getattr(self, '_note_index', 0) + 1
        gt = EXPECTED_NOTE_EXTRACTIONS.get(key, {"symptoms": [], "findings": [], "follow_ups": [], "red_flags": []})
        return ExtractedNote(
            date=note.date, note_type=note.note_type,
            symptoms=gt["symptoms"], findings=gt["findings"],
            follow_ups=gt["follow_ups"], red_flags=gt["red_flags"],
        )


print("✓ NoteExtractor ready (MedGemma 1.5 4B)")

✓ NoteExtractor ready (MedGemma 1.5 4B)


### 5b. Medical Image Analysis

MedGemma 1.5 4B (and thus CCA) supports a wide range of medical imaging modalities:
- **Chest X-rays** — single-image analysis and **longitudinal comparison** (time series)
- **CT and MRI** volumes
- **Histopathology** whole-slide images

#### Longitudinal Comparison (Key Feature)
When CCA detects **2+ images of the same modality** in a patient's chart, it automatically feeds them together into MedGemma as a multi-image input for temporal comparison. This is a powerful continuity-of-care capability: the radiologist who read the most recent CXR may not have had access to the prior, but CCA — with full chart access — can find it and compare.

In [47]:
# =============================================================================
# STAGE 1b: Medical Image Analysis (MedGemma 1.5 4B)
# =============================================================================

IMAGE_ANALYSIS_PROMPT = """Analyze this medical image and extract structured findings.

IMAGE METADATA:
- Modality: {modality}
- Indication: {indication}

Return JSON with:
{{
  "findings": [
    {{"finding": "name", "location": "anatomical location", "size": "if applicable", "confidence": "high/medium/low"}}
  ],
  "impression": "overall interpretation",
  "follow_up_needed": true/false,
  "follow_up_recommendation": "specific recommendation if needed",
  "urgency": "routine/soon/urgent"
}}

Be specific about locations and measurements. Respond with ONLY the JSON object, no explanation."""

# --- Longitudinal comparison prompt (multi-image input) ---
LONGITUDINAL_COMPARISON_PROMPT = """You are comparing multiple {modality} images from the same patient taken at different times.
Carefully examine both images and identify any changes between them.

Image timeline:
{timeline}

For each finding, indicate whether it is new, stable, progressed, or resolved compared to the prior image(s).

Return JSON with:
{{
  "comparison_findings": [
    {{"finding": "description of finding", "change": "new|stable|progressed|resolved", "clinical_significance": "high|medium|low"}}
  ],
  "overall_assessment": "1-2 sentence summary of longitudinal changes",
  "urgency": "routine|soon|urgent"
}}

Respond with ONLY the JSON object, no explanation."""


@dataclass
class ImageAnalysis:
    """Structured analysis from a medical image."""
    modality: str
    date: datetime
    findings: list[dict]  # {"finding": str, "location": str, "confidence": float}
    follow_up_needed: bool = False
    urgency: str = "routine"  # routine, soon, urgent

@dataclass
class LongitudinalComparison:
    """Structured comparison of multiple same-modality images over time."""
    modality: str
    dates: list[datetime]
    image_urls: list[Optional[str]]
    comparison_findings: list[dict]  # {"finding": str, "change": str, "clinical_significance": str}
    overall_assessment: str
    urgency: str = "routine"


def _has_positive_mention(keyword: str, text: str) -> bool:
    """Check if keyword appears as a positive finding (not negated by 'no', 'without', etc.)."""
    # Check for negation patterns: "no nodules", "no ... nodule", "without nodule", "denies nodule"
    negation_pattern = rf'(?:no|without|denies|absent|negative for)\s+(?:\w+[\s,]+)*{keyword}'
    if re.search(negation_pattern, text):
        return False
    return keyword in text


class ImageAnalyzer:
    """
    MedGemma 1.5 4B for medical image analysis.
    
    MedGemma 1.5 4B supports:
    - Chest X-rays with longitudinal comparison
    - CT and MRI volume interpretation
    - Histopathology whole-slide images
    - Anatomical localization
    """
    
    def __init__(self, model=None, processor=None):
        self.use_medgemma = model is not None
        self.model = model
        self.processor = processor
    
    # ----- Single image analysis -----
    
    def analyze(self, imaging: Imaging) -> ImageAnalysis:
        """Analyze a medical image study."""
        if self.use_medgemma:
            return self._analyze_with_model(imaging)
        else:
            return self._mock_analyze(imaging)
    
    def _analyze_with_model(self, imaging: Imaging, image=None) -> ImageAnalysis:
        """Real model inference with MedGemma 1.5 4B."""
        import torch
        from PIL import Image
        
        prompt = IMAGE_ANALYSIS_PROMPT.format(
            modality=imaging.modality,
            indication=imaging.indication,
        )
        
        # Build message content - include image if provided
        content = []
        if image is not None:
            content.append({"type": "image", "image": image})
        elif imaging.image_url:
            content.append({"type": "image", "image": self._load_image(imaging.image_url)})
        content.append({"type": "text", "text": prompt})
        
        messages = [{"role": "user", "content": content}]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device, dtype=torch.bfloat16)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(**inputs, max_new_tokens=1200, do_sample=False)
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)
        
        data = extract_json(response)
        return ImageAnalysis(
            modality=imaging.modality,
            date=imaging.date,
            findings=data.get("findings", []),
            follow_up_needed=data.get("follow_up_needed", False),
            urgency=data.get("urgency", "routine"),
        )
    

    @staticmethod
    def _load_image(url: str):
        """Load a PIL Image from a URL."""
        from PIL import Image
        import requests
        return Image.open(requests.get(url, headers={"User-Agent": "CCA-Pipeline"}, stream=True).raw)
    
    def compare_longitudinal(self, imaging_group: list[Imaging]) -> Optional[LongitudinalComparison]:
        """
        Compare multiple same-modality images longitudinally.
        
        MedGemma 1.5 4B supports multi-image input for chest X-ray time series.
        Images are passed chronologically as separate image tokens in a single prompt.
        """
        if len(imaging_group) < 2:
            return None
        
        # Sort chronologically
        group = sorted(imaging_group, key=lambda x: x.date)
        
        if self.use_medgemma:
            return self._compare_with_model(group)
        else:
            return self._mock_compare(group)
    
    def _compare_with_model(self, group: list[Imaging]) -> LongitudinalComparison:
        """Real multi-image longitudinal comparison with MedGemma 1.5 4B."""
        import torch
        
        # Build timeline description
        timeline_parts = []
        for i, img in enumerate(group, 1):
            timeline_parts.append(f"- Image {i}: {img.date.strftime('%Y-%m-%d')} — {img.indication}")
        timeline = "\n".join(timeline_parts)
        
        prompt = LONGITUDINAL_COMPARISON_PROMPT.format(
            modality=group[0].modality,
            timeline=timeline,
        )
        
        # Build multi-image content: each image followed by the text prompt
        content = []
        for img in group:
            if img.image_url:
                content.append({"type": "image", "image": self._load_image(img.image_url)})
        content.append({"type": "text", "text": prompt})
        
        messages = [{"role": "user", "content": content}]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device, dtype=torch.bfloat16)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(**inputs, max_new_tokens=1200, do_sample=False)
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)
        data = extract_json(response)
        
        return LongitudinalComparison(
            modality=group[0].modality,
            dates=[img.date for img in group],
            image_urls=[img.image_url for img in group],
            comparison_findings=data.get("comparison_findings", []),
            overall_assessment=data.get("overall_assessment", ""),
            urgency=data.get("urgency", "routine"),
        )
    
    def set_patient(self, patient_id: str):
        """Set current patient context for ground truth lookup."""
        self._patient_id = patient_id
        self._image_index = 0

    def _mock_analyze(self, imaging: Imaging) -> ImageAnalysis:
        """Look up expected output from ground truth."""
        key = (getattr(self, '_patient_id', ''), getattr(self, '_image_index', 0))
        self._image_index = getattr(self, '_image_index', 0) + 1
        gt = EXPECTED_IMAGE_ANALYSES.get(key, {"findings": [], "follow_up_needed": False, "urgency": "routine"})
        return ImageAnalysis(
            modality=imaging.modality, date=imaging.date,
            findings=gt["findings"], follow_up_needed=gt["follow_up_needed"], urgency=gt["urgency"],
        )

    def _mock_compare(self, group: list[Imaging]) -> LongitudinalComparison:
        """Look up expected output from ground truth."""
        key = (getattr(self, '_patient_id', ''), group[0].modality)
        gt = EXPECTED_LONGITUDINAL_COMPARISONS.get(key)
        if gt is None:
            return LongitudinalComparison(
                modality=group[0].modality, dates=[img.date for img in group],
                image_urls=[img.image_url for img in group],
                comparison_findings=[{"finding": "No significant interval change", "change": "stable", "clinical_significance": "low"}],
                overall_assessment=f"No significant interval change on {group[0].modality}.", urgency="routine",
            )
        return LongitudinalComparison(
            modality=group[0].modality, dates=[img.date for img in group],
            image_urls=[img.image_url for img in group],
            comparison_findings=gt["comparison_findings"],
            overall_assessment=gt["overall_assessment"], urgency=gt["urgency"],
        )


print("✓ ImageAnalyzer ready (MedGemma 1.5 4B)")

✓ ImageAnalyzer ready (MedGemma 1.5 4B)


### 5c. Lab Trend Analysis (Deterministic)

Labs are already structured data—no model needed. We compute trends, flags, and clinical significance.

In [48]:
# =============================================================================
# STAGE 1c: Lab Trend Analysis (Deterministic)
# =============================================================================

@dataclass
class LabTrendAnalysis:
    """Analysis of laboratory trends."""
    lab_name: str
    values: list[tuple[datetime, float]]  # (date, value) pairs
    trend: str  # "stable", "increasing", "decreasing"
    is_abnormal: bool
    clinical_flag: Optional[str] = None  # e.g., "iron deficiency pattern"

def analyze_lab_trends(patient: Patient) -> list[LabTrendAnalysis]:
    """Analyze all lab trends for a patient."""
    analyses = []
    
    # Group labs by name
    lab_groups = {}
    for lab in patient.labs:
        if lab.name not in lab_groups:
            lab_groups[lab.name] = []
        lab_groups[lab.name].append(lab)
    
    for lab_name, labs in lab_groups.items():
        labs_sorted = sorted(labs, key=lambda x: x.date)
        values = [(l.date, l.value) for l in labs_sorted]
        
        # Determine trend
        if len(values) >= 2:
            first_val, last_val = values[0][1], values[-1][1]
            if all(values[i][1] >= values[i+1][1] for i in range(len(values)-1)):
                trend = "decreasing"
            elif all(values[i][1] <= values[i+1][1] for i in range(len(values)-1)):
                trend = "increasing"
            elif abs(last_val - first_val) / max(first_val, 0.01) > 0.15:
                trend = "increasing" if last_val > first_val else "decreasing"
            else:
                trend = "stable"
        else:
            trend = "insufficient data"
        
        # Check if abnormal
        is_abnormal = any(l.is_abnormal for l in labs_sorted)
        
        # Clinical flags
        clinical_flag = None
        if lab_name == "Hemoglobin" and trend == "decreasing":
            mcv_labs = [l for l in patient.labs if l.name == "MCV"]
            ferritin_labs = [l for l in patient.labs if l.name == "Ferritin"]
            if mcv_labs and mcv_labs[-1].value < 80:
                if ferritin_labs and ferritin_labs[-1].value < 30:
                    clinical_flag = "Iron deficiency anemia pattern"
                else:
                    clinical_flag = "Microcytic anemia - evaluate iron studies"
        
        if lab_name == "Creatinine" and trend == "increasing":
            values_only = [v[1] for v in values]
            if len(values_only) >= 2 and values_only[-1] > values_only[-2] * 1.2:
                clinical_flag = "Acute kidney injury (>20% rise)"
        
        if lab_name == "Calcium" and labs_sorted[-1].value > 10.5:
            pth_labs = [l for l in patient.labs if l.name == "PTH"]
            if pth_labs and pth_labs[-1].value < 25:
                clinical_flag = "Hypercalcemia with suppressed PTH - evaluate for malignancy"
        
        analyses.append(LabTrendAnalysis(
            lab_name=lab_name,
            values=values,
            trend=trend,
            is_abnormal=is_abnormal,
            clinical_flag=clinical_flag,
        ))
    
    return analyses

print("✓ Lab trend analyzer ready")

✓ Lab trend analyzer ready


### 5d. Aggregation: Building the Clinical Context

Combine all extracted data into a structured context for MedGemma synthesis.

In [49]:
# =============================================================================
# STAGE 1d: Aggregation - Building Clinical Context
# =============================================================================

from collections import defaultdict

@dataclass
class AggregatedContext:
    """Complete extracted context ready for MedGemma synthesis."""
    patient: Patient
    note_extractions: list[ExtractedNote]
    image_analyses: list[ImageAnalysis]
    lab_trends: list[LabTrendAnalysis]
    longitudinal_comparisons: list[LongitudinalComparison] = field(default_factory=list)
    
    def to_synthesis_prompt(self) -> str:
        """Format aggregated context for MedGemma 1.5 4B synthesis."""
        
        # Patient demographics
        sections = [
            f"# Patient: {self.patient.name}",
            f"**Demographics:** {self.patient.age}yo {self.patient.sex}",
            f"**Problems:** {', '.join(self.patient.problems)}",
            f"**Medications:** {', '.join(f'{m.name} {m.dose}' for m in self.patient.medications)}",
            f"**Smoking:** {self.patient.smoking_status}",
            f"**Family History:** {', '.join(self.patient.family_history) or 'None'}",
        ]
        
        # Extracted note findings
        sections.append("\n## Extracted from Clinical Notes (MedGemma 1.5 4B)")
        for ext in sorted(self.note_extractions, key=lambda x: x.date):
            sections.append(f"\n### {ext.date.strftime('%Y-%m-%d')} - {ext.note_type}")
            if ext.symptoms:
                sections.append("**Symptoms:**")
                for s in ext.symptoms:
                    sections.append(f"  - {s['symptom']}: {s.get('severity', '')} severity, {s.get('trend', '')} ({s.get('duration', '')})")
            if ext.findings:
                sections.append("**Findings:**")
                for f in ext.findings:
                    sections.append(f"  - {f['finding']}: {f.get('value', '')} [{f.get('interpretation', '')}]")
            if ext.follow_ups:
                sections.append("**Follow-up Items:**")
                for f in ext.follow_ups:
                    status = "completed" if f.get('completed') else "NOT completed"
                    sections.append(f"  - {f['item']} ({f.get('timeframe', '')}) - {status}")
            if ext.red_flags:
                sections.append("**Red Flags:**")
                for rf in ext.red_flags:
                    sections.append(f"  - {rf}")
        
        # Image analysis results
        if self.image_analyses:
            sections.append("\n## Image Analysis (MedGemma 1.5 4B)")
            for img in self.image_analyses:
                sections.append(f"\n### {img.date.strftime('%Y-%m-%d')} - {img.modality}")
                for f in img.findings:
                    raw_conf = f.get('confidence', 0)
                    if isinstance(raw_conf, str):
                        conf_str = raw_conf
                    else:
                        conf_str = f"{float(raw_conf) * 100:.0f}%"
                    sections.append(f"  - {f['finding']} at {f.get('location', 'unspecified')} (confidence: {conf_str})")
                    if 'fleischner_category' in f:
                        sections.append(f"    Fleischner category: {f['fleischner_category']}")
                if img.follow_up_needed:
                    sections.append(f"  **Follow-up needed:** urgency = {img.urgency}")
        
        # Longitudinal image comparisons
        if self.longitudinal_comparisons:
            sections.append("\n## Longitudinal Image Comparison (MedGemma 1.5 4B)")
            for comp in self.longitudinal_comparisons:
                date_range = f"{comp.dates[0].strftime('%Y-%m-%d')} → {comp.dates[-1].strftime('%Y-%m-%d')}"
                sections.append(f"\n### {comp.modality}: {date_range}")
                sections.append(f"**Overall:** {comp.overall_assessment}")
                for cf in comp.comparison_findings:
                    sig = cf.get('clinical_significance', 'unknown')
                    sections.append(f"  - [{cf['change'].upper()}] {cf['finding']} (significance: {sig})")
                sections.append(f"  **Urgency:** {comp.urgency}")
        
        # Lab trends
        sections.append("\n## Lab Trend Analysis")
        for lt in self.lab_trends:
            flag = f" {lt.clinical_flag}" if lt.clinical_flag else ""
            abnormal = " [ABNORMAL]" if lt.is_abnormal else ""
            values_str = " -> ".join(f"{v[1]}" for v in lt.values[-4:])  # Last 4 values
            sections.append(f"- **{lt.lab_name}:** {values_str} (trend: {lt.trend}){abnormal}{flag}")
        
        return "\n".join(sections)

def run_stage1_extraction(patient: Patient, note_extractor: NoteExtractor, image_analyzer: ImageAnalyzer) -> AggregatedContext:
    """Run Stage 1: Extract and analyze all patient data with MedGemma 1.5 4B."""
    
    # Extract from notes
    # Set patient context for ground truth lookup
    note_extractor.set_patient(patient.id)
    image_analyzer.set_patient(patient.id)

    note_extractions = [note_extractor.extract(note) for note in patient.notes]
    
    # Analyze individual images
    image_analyses = [image_analyzer.analyze(img) for img in patient.imaging]
    
    # Longitudinal comparison: group images by modality, compare when 2+ exist
    longitudinal_comparisons = []
    modality_groups = defaultdict(list)
    for img in patient.imaging:
        modality_groups[img.modality].append(img)
    
    for modality, group in modality_groups.items():
        if len(group) >= 2:
            comparison = image_analyzer.compare_longitudinal(group)
            if comparison:
                longitudinal_comparisons.append(comparison)
    
    # Analyze lab trends (deterministic)
    lab_trends = analyze_lab_trends(patient)
    
    return AggregatedContext(
        patient=patient,
        note_extractions=note_extractions,
        image_analyses=image_analyses,
        lab_trends=lab_trends,
        longitudinal_comparisons=longitudinal_comparisons,
    )

print("✓ Aggregation pipeline ready (with longitudinal comparison)")

✓ Aggregation pipeline ready (with longitudinal comparison)


In [50]:
# Initialize Stage 1 extractors (shared MedGemma 1.5 4B instance)
note_extractor = NoteExtractor(model=MEDGEMMA_MODEL, processor=MEDGEMMA_PROCESSOR)
image_analyzer = ImageAnalyzer(model=MEDGEMMA_MODEL, processor=MEDGEMMA_PROCESSOR)

print("✓ Stage 1 pipeline initialized")
print(f"   • NoteExtractor: {'MedGemma 1.5 4B' if USE_MEDGEMMA else 'Mock'}")
print(f"   • ImageAnalyzer: {'MedGemma 1.5 4B' if USE_MEDGEMMA else 'Mock'}")
print(f"   • LabAnalyzer: Deterministic (structured data)")

✓ Stage 1 pipeline initialized
   • NoteExtractor: Mock
   • ImageAnalyzer: Mock
   • LabAnalyzer: Deterministic (structured data)


---
## 5. Safety Rules (Deterministic)

A minimal set of **high-confidence safety rules** that catch clear-cut contraindications. These supplement MedGemma's analysis, and could be expanded, revised, or replaced with additional AI models in future versions. 

> **Why use any deterministic rules?** Drug-disease interactions like NSAID+CKD have near-100% precision and zero ambiguity. Running them deterministically ensures we never miss these critical safety issues, even if MedGemma were to hallucinate or miss them.

In [51]:
# Safety rules: High-confidence contraindications only
NSAIDS = ["ibuprofen", "naproxen", "meloxicam", "diclofenac", "celecoxib", "ketorolac", "indomethacin"]

def check_nsaid_in_ckd(patient: Patient) -> Optional[Alert]:
    """
    CRITICAL SAFETY RULE: NSAID use in patients with CKD.
    
    This is a clear-cut contraindication with no ambiguity—NSAIDs cause 
    acute kidney injury in CKD patients. We use a deterministic rule here
    because precision is 100% and we cannot afford to miss it.
    """
    has_ckd = any("ckd" in p.lower() or "chronic kidney" in p.lower() for p in patient.problems)
    if not has_ckd:
        return None
    
    nsaid = patient.has_medication_class(NSAIDS)
    if not nsaid:
        return None
    
    # Check for worsening renal function (strengthens the alert)
    cr_trend = patient.get_lab_trend("Creatinine")
    worsening = ""
    if len(cr_trend) >= 2:
        recent, previous = cr_trend[-1].value, cr_trend[-2].value
        if recent > previous * 1.2:  # >20% rise
            worsening = f" Creatinine has risen from {previous} to {recent} mg/dL since starting NSAID."
    
    return Alert(
        severity=AlertSeverity.CRITICAL,
        headline="NSAID Contraindicated in CKD",
        summary=f"{nsaid.name} was started {(TODAY - nsaid.start_date).days} days ago by {nsaid.prescriber}. "
                f"Patient has documented CKD—NSAIDs can cause acute kidney injury.{worsening}",
        evidence=[
            f"Diagnosis: CKD on problem list",
            f"Medication: {nsaid.name} {nsaid.dose} started {nsaid.start_date.strftime('%Y-%m-%d')}",
            f"Prescriber: {nsaid.prescriber}",
            f"Current creatinine: {cr_trend[-1].value if cr_trend else 'N/A'} mg/dL",
        ],
        recommendations=[
            "Discontinue NSAID immediately",
            "Contact prescribing physician about alternatives",
            "Recheck creatinine in 3-5 days",
            "Consider acetaminophen or topical therapies",
        ],
    )

def run_safety_rules(patient: Patient) -> list[Alert]:
    """Run deterministic safety rules. Returns high-confidence alerts only."""
    alerts = []
    
    # Add more safety rules here as needed (e.g., metformin + contrast, K+ sparing diuretic + ACE-I)
    nsaid_alert = check_nsaid_in_ckd(patient)
    if nsaid_alert:
        alerts.append(nsaid_alert)
    
    return alerts

print("✓ Safety rules ready (NSAID+CKD)")

✓ Safety rules ready (NSAID+CKD)


### Future Extension: Medication Reconciliation

MedGemma's note extraction could also cross-reference medications mentioned in clinical notes against the patient's formal medication list. This would flag discrepancies such as:

- **Undocumented medications**: OTC drugs, supplements, or prescriptions from outside providers mentioned by the patient but not in the EHR
- **Discontinued medications**: Drugs still on the active list that the patient reports stopping
- **Dosage mismatches**: Patient-reported doses that differ from what's prescribed

This reconciliation step would strengthen the safety rules by ensuring they operate on a complete, accurate medication picture.

---
## 7. Stage 2: MedGemma 1.5 4B Alert Synthesis

In CCA's second stage, MedGemma receives a structured prompt with the ingested longitudinal patient data and returns "Don't Miss" alerts with evidence chains.

In [52]:
# =============================================================================
# MedGemma 1.5 4B Alert Synthesis
# =============================================================================

ALERT_SYNTHESIS_PROMPT = """You are a clinical decision support system helping primary care physicians identify "Don't Miss" findings.

You have been provided with structured extractions from a patient's medical record, all processed by MedGemma 1.5 4B:
- Clinical notes → symptoms, findings, follow-up items
- Medical images → abnormalities, measurements
- Longitudinal image comparisons → temporal changes across same-modality images
- Lab trends → computed deterministically from structured data

Your task: Synthesize these findings to identify concerning patterns that a busy physician might miss.

{aggregated_context}

---

## Focus Areas
1. **Slow-building patterns** - Gradual changes across multiple visits suggesting serious disease
2. **Missed follow-ups** - Recommended tests/imaging that were never completed  
3. **Cross-modal connections** - Links between labs, imaging, symptoms that together suggest a diagnosis
4. **Risk factor amplification** - Findings more concerning given patient's specific risk factors
5. **Longitudinal imaging changes** - New or progressing findings detected via image comparison

Return your analysis as JSON:
```json
{{
  "alerts": [
    {{
      "severity": "critical|high|medium",
      "headline": "Brief alert title",
      "summary": "2-3 sentence clinical explanation",
      "evidence": ["Specific finding 1", "Specific finding 2"],
      "recommendations": ["Action 1", "Action 2"]
    }}
  ]
}}
```

Be specific. Reference actual values, dates, and model outputs from the aggregated context. Respond with ONLY the JSON object, no explanation or reasoning."""


class MedGemmaSynthesizer:
    """
    MedGemma 1.5 4B for alert synthesis.
    
    Uses the same model as Stage 1 extraction, but for a different task:
    synthesizing cross-modal patterns into actionable clinical alerts.
    """
    
    def __init__(self, model=None, processor=None):
        self.use_medgemma = model is not None
        self.model = model
        self.processor = processor
    
    def generate_alerts(self, context: AggregatedContext) -> list[Alert]:
        """Generate alerts from aggregated context."""
        if self.use_medgemma:
            return self._generate_with_model(context)
        else:
            return self._mock_generate(context.patient)
    
    def _generate_with_model(self, context: AggregatedContext) -> list[Alert]:
        """Real model inference with CoT suppression and robust parsing."""
        import torch
        
        prompt = ALERT_SYNTHESIS_PROMPT.format(
            aggregated_context=context.to_synthesis_prompt()
        )
        
        messages = [{"role": "user", "content": [{"type": "text", "text": prompt}]}]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(
                **inputs, 
                max_new_tokens=2000, 
                do_sample=True, 
                temperature=0.1,
                top_p=0.95,
                pad_token_id=self.processor.tokenizer.eos_token_id,
                # This helps prevent the model from getting stuck in long CoT loops
                repetition_penalty=1.1 
            )
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)

        return self._parse_alerts(response)
    
    def _parse_alerts(self, response: str) -> list[Alert]:
        """Extract and parse alerts from model response."""
        severity_map = {"critical": AlertSeverity.CRITICAL, "high": AlertSeverity.HIGH, "medium": AlertSeverity.MEDIUM}
        alerts = []
        for a in extract_json(response).get("alerts", []):
            alerts.append(Alert(
                severity=severity_map.get(a.get("severity", "medium").lower(), AlertSeverity.MEDIUM),
                headline=a.get("headline", "Alert"),
                summary=a.get("summary", ""),
                evidence=a.get("evidence", []),
                recommendations=a.get("recommendations", []),
            ))
        return alerts
    
    def _mock_generate(self, patient: Patient) -> list[Alert]:
        """Look up expected alerts from ground truth."""
        severity_map = {"critical": AlertSeverity.CRITICAL, "high": AlertSeverity.HIGH, "medium": AlertSeverity.MEDIUM}
        return [
            Alert(
                severity=severity_map[a["severity"]],
                headline=a["headline"], summary=a["summary"],
                evidence=a["evidence"], recommendations=a["recommendations"],
            )
            for a in EXPECTED_ALERTS.get(patient.id, [])
        ]


# Initialize synthesizer (same shared MedGemma instance)
synthesizer = MedGemmaSynthesizer(model=MEDGEMMA_MODEL, processor=MEDGEMMA_PROCESSOR)

print("✓ MedGemmaSynthesizer ready")

✓ MedGemmaSynthesizer ready


---
## Patient 1: Maria Klein
### "The Slow-Building Pattern"

Maria is a 58-year-old woman who has been seen multiple times over 8 months. Each visit seemed routine, but connecting the dots reveals a concerning pattern: progressive iron deficiency anemia with weight loss and GI symptoms—a presentation that warrants urgent evaluation for GI malignancy.

In [53]:
show_patient(maria)

In [54]:
# === CCA FULL PIPELINE: Maria Klein ===
print("="*70)
print("CCA MULTI-MODEL PIPELINE")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STAGE 1: Extraction & Analysis
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐")

# 1a. Note extraction (MedGemma 1.5 4B)
print("\n  📝 MedGemma 1.5 4B: Extracting from clinical notes...")
context = run_stage1_extraction(maria, note_extractor, image_analyzer)

for ext in context.note_extractions:
    print(f"\n     {ext.date.strftime('%Y-%m-%d')} - {ext.note_type}:")
    if ext.symptoms:
        print(f"       Symptoms: {', '.join(s['symptom'] for s in ext.symptoms)}")
    if ext.findings:
        print(f"       Findings: {', '.join(f['finding'] for f in ext.findings)}")
    if ext.red_flags:
        print(f"       🚨 Red flags: {len(ext.red_flags)}")

# 1b. Lab analysis (deterministic)
print("\n  📊 Lab Trend Analysis:")
for lt in context.lab_trends:
    if lt.clinical_flag:
        print(f"       {lt.lab_name}: {lt.trend} 🚨 {lt.clinical_flag}")
    elif lt.is_abnormal:
        print(f"       {lt.lab_name}: {lt.trend} [ABNORMAL]")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# STAGE 2: Synthesis (MedGemma 1.5 4B)
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 2: MedGemma 1.5 4B Synthesis ──────────────────────────┐")

# Show aggregated context size
synthesis_prompt = context.to_synthesis_prompt()
print(f"\n  📋 Aggregated context: {len(synthesis_prompt)} characters")
print(f"     • {len(context.note_extractions)} notes processed")
print(f"     • {len(context.lab_trends)} lab trends analyzed")
print(f"     • {len(context.image_analyses)} images analyzed")

print("\n  🧠 MedGemma 1.5 4B: Synthesizing alerts...")
medgemma_alerts = synthesizer.generate_alerts(context)
print(f"     → Generated {len(medgemma_alerts)} alert(s)")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# Safety Rules
# ─────────────────────────────────────────────────────────────────────
print("\n⚡ Safety Rules: ", end="")
safety_alerts = run_safety_rules(maria)
print(f"{len(safety_alerts)} alert(s)")

# ─────────────────────────────────────────────────────────────────────
# Display Alerts
# ─────────────────────────────────────────────────────────────────────
all_alerts = medgemma_alerts + safety_alerts

print("\n" + "="*70)
print("DON'T MISS ALERTS")
print("="*70)
for alert in all_alerts:
    show_alert(alert)

CCA MULTI-MODEL PIPELINE

┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐

  📝 MedGemma 1.5 4B: Extracting from clinical notes...

     2025-06-10 - Annual Physical:
       Symptoms: fatigue

     2025-10-08 - Office Visit:
       Symptoms: fatigue, unintentional weight loss, bloating
       Findings: anemia
       🚨 Red flags: 1

     2026-01-22 - Office Visit:
       Symptoms: constipation, bloating, unintentional weight loss
       Findings: anemia, stool guaiac
       🚨 Red flags: 1

  📊 Lab Trend Analysis:
       Hemoglobin: decreasing 🚨 Iron deficiency anemia pattern
       MCV: decreasing [ABNORMAL]
       Ferritin: insufficient data [ABNORMAL]
       Iron: insufficient data [ABNORMAL]

└────────────────────────────────────────────────────────────────┘

┌─ STAGE 2: MedGemma 1.5 4B Synthesis ──────────────────────────┐

  📋 Aggregated context: 1724 characters
     • 3 notes processed
     • 5 lab trends analyzed
     • 0 images analyzed

  🧠 MedGemma 1.5 4B: Sy

---
## Patient 2: James Morrison
### "The Missed Follow-up" + Longitudinal Image Comparison

James had a left hilar mass incidentally found on a pre-op chest X-ray 4+ months ago. The recommended follow-up CT with contrast was never completed. He's now presenting with hypercalcemia with suppressed PTH — a pattern concerning for malignancy.

**This case demonstrates longitudinal image comparison**: James had a normal chest X-ray ~15 months ago. The radiologist who read the recent CXR noted "no prior imaging available for comparison" — but CCA, with full chart access, finds the prior and feeds both images to MedGemma, confirming the mass is **new**. This is exactly the kind of continuity gap CCA is designed to close.

In [55]:
show_patient(james)

In [56]:
# === CCA FULL PIPELINE: James Morrison ===
print("="*70)
print("CCA PIPELINE: James Morrison")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STAGE 1: Extraction & Analysis
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐")

print("\n  📝 MedGemma 1.5 4B: Extracting from clinical notes...")
context = run_stage1_extraction(james, note_extractor, image_analyzer)

for ext in context.note_extractions:
    print(f"\n     {ext.date.strftime('%Y-%m-%d')} - {ext.note_type}:")
    if ext.symptoms:
        print(f"       Symptoms: {', '.join(s['symptom'] for s in ext.symptoms)}")
    if ext.findings:
        print(f"       Findings: {', '.join(f['finding'] for f in ext.findings)}")
    if ext.red_flags:
        print(f"       Red flags: {len(ext.red_flags)}")

# Show individual image analyses
print(f"\n  🖼️  MedGemma 1.5 4B: Analyzed {len(context.image_analyses)} image(s)")
for img in context.image_analyses:
    for f in img.findings:
        print(f"       {img.date.strftime('%Y-%m-%d')}: {f['finding']} at {f.get('location', '?')}")

# Show longitudinal comparison (the key feature!)
if context.longitudinal_comparisons:
    print(f"\n  🔄 MedGemma 1.5 4B: Longitudinal comparison detected!")
    for comp in context.longitudinal_comparisons:
        date_range = f"{comp.dates[0].strftime('%Y-%m-%d')} → {comp.dates[-1].strftime('%Y-%m-%d')}"
        print(f"       {comp.modality}: {date_range} ({len(comp.comparison_findings)} finding(s))")
        for cf in comp.comparison_findings:
            print(f"       ↳ [{cf['change'].upper()}] {cf['finding']}")
        print(f"       Assessment: {comp.overall_assessment}")

# Lab analysis
print("\n  📊 Lab Trend Analysis:")
for lt in context.lab_trends:
    if lt.clinical_flag:
        print(f"       {lt.lab_name}: {lt.trend} 🚨 {lt.clinical_flag}")
    elif lt.is_abnormal:
        print(f"       {lt.lab_name}: {lt.trend} [ABNORMAL]")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# STAGE 2: Synthesis (MedGemma 1.5 4B)
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 2: MedGemma 1.5 4B Synthesis ──────────────────────────┐")

synthesis_prompt = context.to_synthesis_prompt()
print(f"\n  📋 Aggregated context: {len(synthesis_prompt)} characters")
print(f"     • {len(context.note_extractions)} notes processed")
print(f"     • {len(context.lab_trends)} lab trends analyzed")
print(f"     • {len(context.image_analyses)} images analyzed")
print(f"     • {len(context.longitudinal_comparisons)} longitudinal comparison(s)")

print("\n  🧠 MedGemma 1.5 4B: Synthesizing alerts...")
medgemma_alerts = synthesizer.generate_alerts(context)
print(f"     → Generated {len(medgemma_alerts)} alert(s)")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# Safety Rules + Display
# ─────────────────────────────────────────────────────────────────────
safety_alerts = run_safety_rules(james)

print("\n" + "="*70)
print("DON'T MISS ALERTS")
print("="*70)
for alert in (medgemma_alerts + safety_alerts):
    show_alert(alert)

CCA PIPELINE: James Morrison

┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐

  📝 MedGemma 1.5 4B: Extracting from clinical notes...

     2025-09-28 - Pre-op Visit:
       Findings: left hilar mass
       Red flags: 1

     2025-12-22 - Urgent Care:
       Symptoms: cough

     2026-01-22 - Office Visit:

  🖼️  MedGemma 1.5 4B: Analyzed 2 image(s)
       2024-11-02: no acute abnormality at chest
       2025-09-28: pulmonary mass at lung

  🔄 MedGemma 1.5 4B: Longitudinal comparison detected!
       Chest X-ray: 2024-11-02 → 2025-09-28 (1 finding(s))
       ↳ [NEW] New pulmonary mass, not present on prior imaging
       Assessment: Comparison of Chest X-ray images 330 days apart reveals 1 new finding(s) not present on prior study. Clinical correlation and further workup recommended.

  📊 Lab Trend Analysis:
       Calcium: increasing 🚨 Hypercalcemia with suppressed PTH - evaluate for malignancy
       HbA1c: insufficient data [ABNORMAL]

└───────────────────────────

---
## Patient 3: Dorothy Williams
### "The Drug-Disease Interaction"

Dorothy has well-documented CKD Stage 3a. She was recently started on an NSAID by orthopedics for knee OA.

**This case demonstrates the hybrid approach**: The NSAID+CKD interaction is caught by a deterministic safety rule (100% precision), while MedGemma provides additional context about the accelerated renal decline.

In [57]:
show_patient(dorothy)

In [58]:
# Run CCA Pipeline
context = run_stage1_extraction(dorothy, note_extractor, image_analyzer)
medgemma_alerts = synthesizer.generate_alerts(context)
safety_alerts = run_safety_rules(dorothy)

# Safety alerts first (deterministic), then MedGemma synthesis
for alert in (safety_alerts + medgemma_alerts):
    show_alert(alert)

---
## 9. Evaluation: MedGemma vs. Expected Outputs

Does MedGemma catch what matters? We evaluate final alerts on two axes:

1. **Severity threshold** — Does the model flag the concern at *at least* the expected severity level?
2. **Clinical completeness (LLM-as-judge)** — Does the model's alert capture the same core clinical concern as the ground truth, without missing anything critical? MedGemma itself adjudicates with a binary YES/NO verdict. *(Skipped in mock mode.)*

In [59]:
# =============================================================================
# Evaluation Framework
# =============================================================================

SEVERITY_RANK = {"medium": 1, "high": 2, "critical": 3}

JUDGE_PROMPT = """You are a clinical evaluation judge. Compare a model-generated alert against a ground-truth reference alert for the same patient.

GROUND TRUTH ALERT:
  Headline: {gt_headline}
  Severity: {gt_severity}
  Evidence: {gt_evidence}
  Recommendations: {gt_recommendations}

MODEL OUTPUT ALERT:
  Headline: {model_headline}
  Severity: {model_severity}
  Evidence: {model_evidence}
  Recommendations: {model_recommendations}

Does the model's alert capture the same core clinical concern as the ground truth without missing anything critical?
Answer exactly YES or NO, followed by a single sentence of rationale."""


def evaluate_severity(actual: list, patient) -> dict:
    """Check that the model's top alert meets or exceeds the expected severity."""
    gt_list = EXPECTED_ALERTS.get(patient.id, [])
    if not gt_list or not actual:
        return {"check": "Severity threshold", "expected": "—", "actual": "—", "result": "SKIP", "detail": "No alerts to compare"}

    gt_sev = gt_list[0]["severity"]
    # Find the highest-severity alert produced by the model
    model_sev = max(actual, key=lambda a: SEVERITY_RANK.get(a.severity.value, 0)).severity.value
    passed = SEVERITY_RANK.get(model_sev, 0) >= SEVERITY_RANK.get(gt_sev, 0)
    return {
        "check": "Severity threshold",
        "expected": f"{gt_sev.upper()} or higher",
        "actual": model_sev.upper(),
        "result": "PASS" if passed else "FAIL",
        "detail": "",
    }


def evaluate_with_judge(actual: list, patient, model=None, processor=None) -> dict:
    """Use MedGemma as an LLM judge to assess clinical completeness."""
    gt_list = EXPECTED_ALERTS.get(patient.id, [])
    if not gt_list or not actual:
        return {"check": "Clinical completeness", "expected": "—", "actual": "—", "result": "SKIP", "detail": "No alerts to compare"}

    if model is None:
        return {"check": "Clinical completeness", "expected": "—", "actual": "—", "result": "N/A", "detail": "Requires model (skipped in mock mode)"}

    import torch
    gt = gt_list[0]
    act = actual[0]

    prompt = JUDGE_PROMPT.format(
        gt_headline=gt["headline"],
        gt_severity=gt["severity"].upper(),
        gt_evidence="; ".join(gt["evidence"]),
        gt_recommendations="; ".join(gt["recommendations"]),
        model_headline=act.headline,
        model_severity=act.severity.value.upper(),
        model_evidence="; ".join(act.evidence),
        model_recommendations="; ".join(act.recommendations),
    )

    messages = [{"role": "user", "content": [{"type": "text", "text": prompt}]}]
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device)
    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(
            **inputs, max_new_tokens=100, do_sample=False,
            pad_token_id=processor.tokenizer.eos_token_id,
        )
        generation = generation[0][input_len:]

    response = processor.decode(generation, skip_special_tokens=True).strip()
    verdict = "YES" if response.upper().startswith("YES") else "NO"
    return {
        "check": "Clinical completeness",
        "expected": "Core concern captured",
        "actual": verdict,
        "result": verdict,
        "detail": response,
    }


def render_evaluation(patient_name: str, results: list[dict]):
    """Render evaluation results as a styled HTML scorecard."""
    def result_color(result: str) -> str:
        if result in ("PASS", "YES"): return "#22c55e"
        if result == "N/A":           return "#94a3b8"
        return "#ef4444"

    header = f"""
    <div style="font-family: system-ui; margin: 20px 0;">
    <h3 style="margin-bottom: 8px;">Evaluation: {patient_name}</h3>
    <table style="border-collapse: collapse; font-size: 14px; width: 100%; max-width: 700px;">
    <tr style="background: #1a1a2e; color: white;">
        <th style="padding: 8px 12px; text-align: left;">Check</th>
        <th style="padding: 8px 12px; text-align: center;">Expected</th>
        <th style="padding: 8px 12px; text-align: center;">Actual</th>
        <th style="padding: 8px 12px; text-align: center;">Result</th>
    </tr>"""

    body = ""
    for i, row in enumerate(results):
        bg = "#f8f9fa" if i % 2 == 0 else "#ffffff"
        color = result_color(row["result"])
        detail = f'<br><span style="font-size: 12px; color: #666;">{row["detail"]}</span>' if row["detail"] else ""
        body += f"""
    <tr style="background: {bg};">
        <td style="padding: 6px 12px;">{row['check']}{detail}</td>
        <td style="padding: 6px 12px; text-align: center;">{row['expected']}</td>
        <td style="padding: 6px 12px; text-align: center;">{row['actual']}</td>
        <td style="padding: 6px 12px; text-align: center; font-weight: 600; color: {color};">{row['result']}</td>
    </tr>"""

    footer = "</table></div>"
    display(HTML(header + body + footer))


print("\u2713 Evaluation framework loaded")

✓ Evaluation framework loaded


In [60]:
# =============================================================================
# Run Evaluation: All Patients
# =============================================================================

mode_label = "MedGemma 1.5 4B" if USE_MEDGEMMA else "Mock (self-check)"
judge_model = MEDGEMMA_MODEL if USE_MEDGEMMA else None
judge_processor = MEDGEMMA_PROCESSOR if USE_MEDGEMMA else None
print(f"Evaluating pipeline in {'MODEL' if USE_MEDGEMMA else 'MOCK'} mode\n")

for patient in [maria, james, dorothy]:
    context = run_stage1_extraction(patient, note_extractor, image_analyzer)
    medgemma_alerts = synthesizer.generate_alerts(context)
    safety_alerts = run_safety_rules(patient)
    all_alerts = medgemma_alerts + safety_alerts

    results = []
    results.append(evaluate_severity(all_alerts, patient))
    results.append(evaluate_with_judge(all_alerts, patient, judge_model, judge_processor))
    render_evaluation(f"{patient.name} ({mode_label})", results)

Evaluating pipeline in MOCK mode



Check,Expected,Actual,Result
Severity threshold,CRITICAL or higher,CRITICAL,PASS
Clinical completenessRequires model (skipped in mock mode),—,—,N/A


Check,Expected,Actual,Result
Severity threshold,CRITICAL or higher,CRITICAL,PASS
Clinical completenessRequires model (skipped in mock mode),—,—,N/A


Check,Expected,Actual,Result
Severity threshold,HIGH or higher,CRITICAL,PASS
Clinical completenessRequires model (skipped in mock mode),—,—,N/A


---
## 8. Summary and Next Steps
### Integration into the EMR

A physician preparing for a patient visit sees this in their chart: 